In [1]:
import salem 
import xarray as xr
import pandas as pd
#import xesmf as xe
import matplotlib.pyplot as plt
import glob
import numpy as np

In [3]:
ds=xr.open_dataset('/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-11_12:00:00')#.U[16,0,].values
ds

<xarray.Dataset>
Dimensions:                (Time: 1, south_north: 350, west_east: 300,
                            bottom_top: 44, bottom_top_stag: 45,
                            soil_layers_stag: 4, west_east_stag: 301,
                            south_north_stag: 351, dust_erosion_dimension: 3,
                            klevs_for_dust: 1,
                            bio_emissions_dimension_stag: 200, klevs_for_dvel: 1)
Coordinates:
    XLAT                   (Time, south_north, west_east) float32 ...
    XLONG                  (Time, south_north, west_east) float32 ...
    XTIME                  (Time) datetime64[ns] ...
    XLAT_U                 (Time, south_north, west_east_stag) float32 ...
    XLONG_U                (Time, south_north, west_east_stag) float32 ...
    XLAT_V                 (Time, south_north_stag, west_east) float32 ...
    XLONG_V                (Time, south_north_stag, west_east) float32 ...
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag,
                                dust_erosion_dimension, klevs_for_dust,
                                bio_emissions_dimension_stag, klevs_for_dvel
Data variables: (12/237)
    Times                  (Time) |S19 ...
    LU_INDEX               (Time, south_north, west_east) float32 ...
    ZNU                    (Time, bottom_top) float32 ...
    ZNW                    (Time, bottom_top_stag) float32 ...
    ZS                     (Time, soil_layers_stag) float32 ...
    DZS                    (Time, soil_layers_stag) float32 ...
    ...                     ...
    PCB                    (Time, south_north, west_east) float32 ...
    PC                     (Time, south_north, west_east) float32 ...
    LANDMASK               (Time, south_north, west_east) float32 ...
    LAKEMASK               (Time, south_north, west_east) float32 ...
    SST                    (Time, south_north, west_east) float32 ...
    SST_INPUT              (Time, south_north, west_east) float32 ...
Attributes: (12/119)
    TITLE:                            OUTPUT FROM *             PROGRAM:WRF-C...
    START_DATE:                      2024-01-11_12:00:00
    SIMULATION_START_DATE:           2024-01-11_12:00:00
    WEST-EAST_GRID_DIMENSION:        301
    SOUTH-NORTH_GRID_DIMENSION:      351
    BOTTOM-TOP_GRID_DIMENSION:       45
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      -1
    ETAC:                            0.0

In [7]:
def nearest_position( stn_lat, stn_lon, lat2d, lon2d):
    """获取最临近格点坐标索引
    stn_lat  : 站点纬度
    stn_lon  : 站点经度
    lat2d    : numpy.ndarray网格二维经度坐标
    lon2d    : numpy.ndarray网格二维纬度坐标
    Return: (y_index, x_index)
    """
    difflat = stn_lat - lat2d;
    difflon = stn_lon - lon2d;
    rad = np.multiply(difflat,difflat)+np.multiply(difflon , difflon)
    aa=np.where(rad==np.min(rad))
    ind=np.squeeze(np.array(aa))
    return tuple(ind)
lat2d=ds.XLAT[0]
lon2d=ds.XLONG[0]
stn_lat=31.77
stn_lon=117.49
#117.498268E 31.774524N
id1,id2=nearest_position( stn_lat, stn_lon, lat2d, lon2d)

In [8]:
id1,id2

(177, 152)

In [10]:
ds.XLONG[0,id1,id2]

<xarray.DataArray 'XLONG' ()>
array(117.49512, dtype=float32)
Coordinates:
    XLAT     float32 31.77
    XLONG    float32 117.5
    XTIME    datetime64[ns] 2024-01-11T12:00:00
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LONGITUDE, WEST IS NEGATIVE
    units:        degree_east
    stagger:

In [12]:
ds.XLAT[0,id1,id2]

<xarray.DataArray 'XLAT' ()>
array(31.766106, dtype=float32)
Coordinates:
    XLAT     float32 31.77
    XLONG    float32 117.5
    XTIME    datetime64[ns] 2024-01-11T12:00:00
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LATITUDE, SOUTH IS NEGATIVE
    units:        degree_north
    stagger:

In [14]:
ds=salem.open_wrf_dataset('/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-11_12:00:00')

In [20]:
ds.Z

<xarray.DataArray 'Z' (time: 1, bottom_top: 44, south_north: 350, west_east: 300)>
[4620000 values with dtype=float32]
Coordinates:
    lat          (south_north, west_east) float32 30.01 30.01 ... 33.45 33.45
    lon          (south_north, west_east) float32 115.8 115.8 ... 119.2 119.3
    xtime        (time) datetime64[ns] 2024-01-11T12:00:00
  * time         (time) datetime64[ns] 2024-01-11T12:00:00
  * west_east    (west_east) float64 -1.095e+05 -1.085e+05 ... 1.895e+05
  * south_north  (south_north) float64 -9.95e+04 -9.85e+04 ... 2.495e+05
Dimensions without coordinates: bottom_top
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  Full model height
    units:        m
    stagger:      Z
    pyproj_srs:   +proj=lcc +lat_0=31.0000114440918 +lon_0=117 +lat_1=10 +lat...

In [36]:
dssel=ds[['xtime','U','V','Z','TK','lat','lon','bottom_top']].isel(south_north=id1,west_east=id2)



dssel['BJTime']=dssel['time']+pd.Timedelta('8H')#=pd.date_range('20220821T00','20220821T23',freq='1H')+pd.Timedelta('8H')
dssel#['Time']

#dssel=dssel.sel(xtime=dssel.Time.dt.hour.isin(np.arange(8,21,1)))
dssel['wind']=np.sqrt(dssel.U**2+dssel.V**2)
dssel

<xarray.Dataset>
Dimensions:      (time: 1, bottom_top: 44)
Coordinates:
    xtime        (time) datetime64[ns] 2024-01-11T12:00:00
    lat          float32 31.77
    lon          float32 117.5
  * bottom_top   (bottom_top) int64 0 1 2 3 4 5 6 7 ... 36 37 38 39 40 41 42 43
  * time         (time) datetime64[ns] 2024-01-11T12:00:00
    west_east    float64 4.25e+04
    south_north  float64 7.75e+04
Data variables:
    U            (time, bottom_top) float32 -0.00992 0.02844 ... 22.7 21.76
    V            (time, bottom_top) float32 2.255 2.349 2.835 ... 1.382 1.93
    Z            (time, bottom_top) float32 ...
    TK           (time, bottom_top) float32 ...
    BJTime       (time) datetime64[ns] 2024-01-11T20:00:00
    wind         (time, bottom_top) float32 2.255 2.349 2.844 ... 22.74 21.85
Attributes: (12/120)
    TITLE:                            OUTPUT FROM *             PROGRAM:WRF-C...
    START_DATE:                      2024-01-11_12:00:00
    SIMULATION_START_DATE:           2024-01-11_12:00:00
    WEST-EAST_GRID_DIMENSION:        301
    SOUTH-NORTH_GRID_DIMENSION:      351
    BOTTOM-TOP_GRID_DIMENSION:       45
    ...                              ...
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      -1
    ETAC:                            0.0
    pyproj_srs:                      +proj=lcc +lat_0=31.0000114440918 +lon_0...

In [37]:
L=glob.glob('/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_*:00:00')
L.sort()
L

['/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_00:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_01:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_02:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_03:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_04:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_05:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_06:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_07:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_08:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_09:00:00',
 '/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02

In [54]:
def nearest_position( stn_lat, stn_lon, lat2d, lon2d):
    """获取最临近格点坐标索引
    stn_lat  : 站点纬度
    stn_lon  : 站点经度
    lat2d    : numpy.ndarray网格二维经度坐标
    lon2d    : numpy.ndarray网格二维纬度坐标
    Return: (y_index, x_index)
    """
    difflat = stn_lat - lat2d;
    difflon = stn_lon - lon2d;
    rad = np.multiply(difflat,difflat)+np.multiply(difflon , difflon)
    aa=np.where(rad==np.min(rad))
    ind=np.squeeze(np.array(aa))
    return tuple(ind)


DS=[]
for l in L:
    ds=salem.open_wrf_dataset(l)
    

    lat2d=ds.lat
    lon2d=ds.lon
    stn_lat=31.77
    stn_lon=117.49
    #117.498268E 31.774524N
    id1,id2=nearest_position( stn_lat, stn_lon, lat2d, lon2d)    

    dssel=ds[['xtime','U','V','W','Z','TK','lat','lon','bottom_top']].isel(south_north=id1,west_east=id2)



    dssel['BJTime']=dssel['time']+pd.Timedelta('8H')#=pd.date_range('20220821T00','20220821T23',freq='1H')+pd.Timedelta('8H')
    dssel#['Time']

    #dssel=dssel.sel(xtime=dssel.Time.dt.hour.isin(np.arange(8,21,1)))
    dssel['wind']=np.sqrt(dssel.U**2+dssel.V**2)
    dssel
    dssel.load()
    
    DS.append(dssel)
    print(l)

/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_00:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_01:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_02:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_03:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_04:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_05:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_06:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_07:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_08:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_09:00:00
/exports/d4/xuejingkai/WRF/work/LYK/case2024011112/runwrf/wrfout_d02_2024-01-13_10:00:00
/exports/d4/xuejingka

In [55]:
DS2=xr.concat(DS,dim='time')

In [56]:
DS2.Z[0]#.plot()

<xarray.DataArray 'Z' (bottom_top: 44)>
array([   30.354027,    57.83943 ,    85.56023 ,   109.48083 ,
         133.47548 ,   157.52406 ,   193.80856 ,   242.56555 ,
         287.70673 ,   358.34525 ,   488.68652 ,   689.7363  ,
         880.6556  ,  1107.8175  ,  1370.73    ,  1618.7772  ,
        2007.6318  ,  2535.4043  ,  3102.6077  ,  3674.1401  ,
        4261.7427  ,  4871.9136  ,  5486.6733  ,  6129.877   ,
        6805.521   ,  7477.6167  ,  8232.784   ,  9104.597   ,
       10000.458   , 10897.683   , 11780.7     , 12647.761   ,
       13513.694   , 14363.513   , 15151.766   , 15906.43    ,
       16629.836   , 17311.498   , 17960.973   , 18520.46    ,
       19052.43    , 19532.916   , 19951.578   , 20341.896   ],
      dtype=float32)
Coordinates:
    xtime        datetime64[ns] 2024-01-13
    lat          float32 31.77
    lon          float32 117.5
  * bottom_top   (bottom_top) int64 0 1 2 3 4 5 6 7 ... 36 37 38 39 40 41 42 43
    time         datetime64[ns] 2024-01-13
    west_east    float64 4.25e+04
    south_north  float64 7.75e+04
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  Full model height
    units:        m
    stagger:      Z
    pyproj_srs:   +proj=lcc +lat_0=31.0000114440918 +lon_0=117 +lat_1=10 +lat...

In [57]:
DS2.to_netcdf('out.nc')

In [58]:
DS2.to_dataframe().reset_index().to_csv('out.csv')